## **SpDB Manual之系列2：处理Python中原生数据类型**

### **2.1 SpDB为数据访问提供统一的Entry入口。**
- 在SpDB中，Entry是处理结构化树状结构HTree的入口。
- Entry为数据对象提供可灵活游走的指针
- Entry()的参数可以是任意的数据对象，包括Dict,List等    

In [1]:
## 加载基本环境
from spdm.data.Entry import open_entry,Entry
from spdm.utils.logger import logger

In [2]:
### 接收字典
example_dict = {
    'name': 'Alice',
    'age': 25,
    'hobbies': ['reading', 'painting', 'yoga'],
    'address': [
        {
            'street': '123 Main St',
            'city': 'Anytown',
            'state': 'CA',
            'zip': '12345'
        },
        {
            'street': '456 Oak St',
            'city': 'Othertown',
            'state': 'NY',
            'zip': '67890'
        },
        {
            'street': '789 Elm St',
            'city': 'Somewhere',
            'state': 'CO',
            'zip': '24680'
        }
    ],
    'spouse': {
        'spouse_name':
            {
                'name':'Bob',
                'age': 27,
            },
        'name': 'Bob',
        'age': 27,
        'hobbies': ['music', 'skiing', 'reading']
    }
}

### example_dict作为参数传入Entry，生成Entry对象
dict_entry = Entry(example_dict)
### dict_entry是一个Entry对象，可以使用Entry的方法。如.get()方法,获取Entry对象的值
dict_entry.get()

{'name': 'Alice',
 'age': 25,
 'hobbies': ['reading', 'painting', 'yoga'],
 'address': [{'street': '123 Main St',
   'city': 'Anytown',
   'state': 'CA',
   'zip': '12345'},
  {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
  {'street': '789 Elm St',
   'city': 'Somewhere',
   'state': 'CO',
   'zip': '24680'}],
 'spouse': {'spouse_name': {'name': 'Bob', 'age': 27},
  'name': 'Bob',
  'age': 27,
  'hobbies': ['music', 'skiing', 'reading']}}

In [186]:
### 接收list
my_list = [
    [1, 2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    [11, 12, 13],
    [14, 15, 16, 17],
    [18, 19, 20, 21, 22, 23, 24]
]   
### my_list作为参数传入Entry，生成Entry对象   
list_entry = Entry(my_list)
list_entry.get()

[[1, 2, 3, 4, 5],
 [6, 7, 8, 9, 10],
 [11, 12, 13],
 [14, 15, 16, 17],
 [18, 19, 20, 21, 22, 23, 24]]

，并不真正的获取数据，只有当执行get操作时，数据才被真正访问。

### **2.2 SpDB中基本数据操作**

- get()：读取数据
- put(): 写入数据
- keys()：获得关键节点
- count()：计算当前节点的元数个数
其目标对象是path


#### **2.2.1 .get()方法**
- get(path): 读取数据,返回数据对象的值.
    - path是空的时候,返回当前根节点的数据对象
    - path参数是一个路径,路径可以是字符串,也可以是一个包含list索引的字符串的组合。
    - 相当于.child()+.fetch()操作

In [3]:
### 不指定path，返回整个节点的值
dict_entry.get()

{'name': 'Alice',
 'age': 25,
 'hobbies': ['reading', 'painting', 'yoga'],
 'address': [{'street': '123 Main St',
   'city': 'Anytown',
   'state': 'CA',
   'zip': '12345'},
  {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
  {'street': '789 Elm St',
   'city': 'Somewhere',
   'state': 'CO',
   'zip': '24680'}],
 'spouse': {'spouse_name': {'name': 'Bob', 'age': 27},
  'name': 'Bob',
  'age': 27,
  'hobbies': ['music', 'skiing', 'reading']}}

In [6]:
### 指定path，path表示方法1：'key1.key2.key3'。
### address是dict中的一个list，可以使用索引获取list中的元素，如address[0]，address[1]
dict_entry.get('address[0].street')

'123 Main St'

In [7]:
### 指定path，path表示方法2：['key1/key2/key3']。
dict_entry.get('address/0/street')

'123 Main St'

#### **2.2.1 .put()方法**
- put(key:value): 写入数据，返回Entry path对象.
    - key是一个路径,路径可以是字符串,也可以是一个包含list索引的字符串的组合。
    - 输出的path是一个Entry对象，可以继续进行get等操作。
    - 相当于.child()+.update()操作

In [34]:
### 根节点中增加height节点，值为170
dict_entry.put('height','170')

In [35]:
### 获取新增加的height节点的值
dict_entry.get('height')

'170'

In [36]:
### 给address[0]节点增加元素
dict_entry.put('address[0].phone','17756014979')

In [37]:
### 检查iphone节点是否增加成功
dict_entry.get('address[0]')

{'street': '123 Main St',
 'city': 'Anytown',
 'state': 'CA',
 'zip': '12345',
 'phone': '17756014979'}

In [ ]:
### 增加新的address list节点的元数
### **"Not implemented "**
dict_entry.put('address[3]',{'street': '901 Elm St','city': 'Somewhere','state': 'CN','zip': '25780'})

In [21]:
dict_entry.get('address[2]')

{'street': '901 Elm St', 'city': 'Somewhere', 'state': 'CN', 'zip': '25780'}

#### **2.2.1 .keys()方法**
- keys():获得节点关键字

In [58]:
[*dict_entry.keys()]

['name', 'age', 'hobbies', 'address', 'spouse', 'height']

#### **2.2.1 .count方法**
- count:计算当前节点子节点个数

In [8]:
dict_entry.count

{'name': 'Alice',
 'age': 25,
 'hobbies': ['reading', 'painting', 'yoga'],
 'address': [{'street': '123 Main St',
   'city': 'Anytown',
   'state': 'CA',
   'zip': '12345'},
  {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
  {'street': '789 Elm St',
   'city': 'Somewhere',
   'state': 'CO',
   'zip': '24680'}],
 'spouse': {'spouse_name': {'name': 'Bob', 'age': 27},
  'name': 'Bob',
  'age': 27,
  'hobbies': ['music', 'skiing', 'reading']}}

In [9]:
### 计算当前节点的元数个数
print(f'address的元数个数是',dict_entry.child("address").count)
print(f'子节点spouse.spouse_name的元数个数是',dict_entry.child("spouse.spouse_name").count)

address的元数个数是 [{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'}, {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'}, {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}]
子节点spouse.spouse_name的元数个数是 {'name': 'Bob', 'age': 27}


### *2.3 Entry操作支持指标游走：*
- child(path):将指标移动到目标子节点，返回是个entry
- paraent(path):将指标移动到目标父节点，返回是个entry
- next():将指标移动到目标节点内元素的兄弟，返回是个entry
 - 该功能只对元素是List有效

In [15]:
### 指标游走到当前节点"spouse.spouse_name"
child_data = dict_entry.child("spouse.spouse_name")
child_data.fetch()

{'name': 'Bob', 'age': 27}

In [11]:
### 指标移到当前节点的paraent节点，并获取数据
paraent_tree = child_data.parent
paraent_tree.fetch()

{'spouse_name': {'name': 'Bob', 'age': 27},
 'name': 'Bob',
 'age': 27,
 'hobbies': ['music', 'skiing', 'reading']}

In [12]:

dict_entry.child("address[0]").parent.fetch()

[{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'},
 {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
 {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}]

In [17]:
### 当当前节点的元素是LIST时，指标可以移动遍历到其节点内所有value
print(dict_entry.child("address[0]").next().fetch())
print(dict_entry.child("address[1]").next().fetch())


{'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'}
{'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}


### 2.3 对指标path的操作，可灵活支持数据读写和更新：
- fetch(): 获取指标所指向的节点的数据，返回是对应的数值
- insert(value):在指标所指向的数据中插入新的数据，在原有数据基础上增加新的。
- update(value):在指标所指向的数据中更新数据，修改原有的数值。
- for_each():在指标所指向的数据中遍历数据

In [247]:
### for_each用来遍历dict中delist
for v in dict_entry.child("address").for_each():
    print(v)    

(0, {'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'})
(1, {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'})
(2, {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'})


In [14]:
### insert插入增加一个值，使之前的数变成list
dict_entry.child("age").insert(50)
dict_entry.child("age").fetch()

25

In [250]:
dict_entry.child("name").insert({"first":"Alice","last":"Smith"})
## 注意，插入更多数据后，原来的数据被组成了LIST
dict_entry.child("name").fetch()

['Alice', {'first': 'Alice', 'last': 'Smith'}]

In [254]:
### update是更新之前的整个数值
print(f'原来的age是',dict_entry.child("age").fetch())
dict_entry.child("age").update(100)
print(f'现在的age是',dict_entry.child("age").fetch())
# dict_entry.child("age").get()

原来的age是 90
现在的age是 100
